## Carga de datos en base de datos vectorial

Se utilizara la base de datos vectorial de Pinecone

### Procesamiento de documentos

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
# Genero los chunks
def chunkData(docs, chunk_size=100, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(docs)
    return chunks

In [32]:
# Cargo el documento 
filePath = "cv_valentin_pertierra.pdf"

loader = PyPDFLoader(filePath)
docs = loader.load()

# Genero los chunks 
chunks = chunkData(docs, chunk_size=500, chunk_overlap=100)

In [33]:
# Muestro algunos chunks 
for i, chunk in enumerate(chunks):
    if i<3:
        print(f'Chunk {i}:')
        print(chunk, '\n')

Chunk 0:
page_content='PERFIL    _________________________________________  
 
Ingeniero en electrónica con experiencia en el desarrollo de 
aplicaciones para la industria 4.0. Considero que tengo constancia y 
firmeza para alcanzar los objetivos que me propongo, ideando 
soluciones a las dificultades que puedan surgir. Me gusta afrontar 
nuevos desafíos, buscando siempre aprender y mejorar, tanto en lo 
humano, como en lo académico y profesional.  
 
 
 
 
 
EXPERENCIA LABORAL 
 
Analista de Transformación Digital' metadata={'source': 'cv_valentin_pertierra.pdf', 'page': 0} 

Chunk 1:
page_content='EXPERENCIA LABORAL 
 
Analista de Transformación Digital 
Julio 2020 – Actualidad 
 
 
 
 
 
Becario 
Mayo 2018 – Diciembre 2018 
 
 
Pasante 
Abril 2012 – Diciembre 2012 
 
FORMACIÓN ACADÉMICA 
 
Idioma Chino Mandarín 
Septiembre 2019 – Enero 2020 
 
Ingeniería Electrónica 
Marzo 2013 – Abril 2019 
 
Idioma Chino Mandarín 
 Abril 2016 – Agosto 2019 
 
Técnico en Electrónica 
Marzo 2007 – D

### Generación de embbeding y carga en base de datos vectorial

In [7]:
import os
import time
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import HuggingFaceEmbeddings

from dotenv import load_dotenv

In [13]:
load_dotenv()
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")


In [34]:
#Connect to DB Pinecone
pc=Pinecone(api_key=PINECONE_API_KEY)

cloud = 'aws'
region = 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

index_name = 'raglse'
namespace = "espacio"
dimension = 384

In [35]:
pc.list_indexes().names()

['raglse']

In [36]:
# Elimino el indice si es que ya existe en la base de datos
if index_name in pc.list_indexes().names():
  pc.delete_index(index_name)
  print("index {} borrado".format(index_name))

if index_name not in pc.list_indexes().names():
    # Como lo borre en el paso anterior siempre deberia entrar aca
    print("index creado con el nombre: {}".format(index_name))
    pc.create_index(
        index_name,
        dimension=dimension, 
        metric='cosine',
        spec=spec
        )
else:
    print("el index con el nombre {} ya estaba creado".format(index_name))
    

index raglse borrado
index creado con el nombre: raglse


In [37]:
# Cargo un modelo de embeddings compatible
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [38]:
# Cargo chunks en base de datos
docsearch = PineconeVectorStore.from_documents(
    documents=chunks,
    index_name=index_name,
    embedding=embedding_model, 
    namespace=namespace
)
print("upserted values to {} index".format(index_name))

time.sleep(1)

upserted values to raglse index


### Busuquedas en base de datos

Realizo pruebas para verificar que los datos se guardaron correctamente

In [39]:
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [40]:
vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embedding_model,
    namespace=namespace,
)
retriever=vectorstore.as_retriever()

In [42]:
query = "Cuales son los estudios que tienen?"
vectorstore.similarity_search(query, k=3)

[Document(id='396888ff-6fca-4a8b-9910-e2374f9f2cba', metadata={'page': 0.0, 'source': 'cv_valentin_pertierra.pdf'}, page_content='Cursado hasta el nivel 7 \n \nE.T.N°28 “República Francesa”  \nPromedio 9,34 \n \n___________________________________________________________ \n \nExamen First Certificate in English  \nNivel B2 (MCER) acreditado por University of Cambridge ESOL Examinations \n \nExamen HSK3  \nNivel B1 (MCER) acreditado por Hanban \n \n__________________________________________________________\n \n● Desarrollo de soluciones end to end (diseño, \nprototipado, implementación, puesta en marcha)'),
 Document(id='1381192f-7f60-4586-8545-42b1462a3924', metadata={'page': 0.0, 'source': 'cv_valentin_pertierra.pdf'}, page_content='Armado y prueba de muestras (equipos de audio, TV y electrodomésticos). Verificación del \ncumplimiento de las especificaciones técnicas de las muestras. \n \n___________________________________________________________ \n \nBeijing Language and Culture Uni